In [181]:
import pandas as pd
import pyodbc

In [182]:
# Database connection details
server = 'LAPTOP-QJ16KG8Q\SQLEXPRESS'
database = 'test4'
password = '' 
driver = '{SQL SERVER}'  # Use the appropriate driver for your SQL Server version

In [183]:
# Read the CSV file into a DataFrame
df = pd.read_csv('job_clean.csv')

In [184]:
# Database connection
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};PWD={password}')
cursor = conn.cursor()

In [185]:
cursor.execute("""
    CREATE TABLE [Company] (
      [company_id] int IDENTITY(1,1) PRIMARY KEY,
      [name] varchar(255),
    );
""")
conn.commit()

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC SQL Server Driver][SQL Server]There is already an object named 'Company' in the database. (2714) (SQLExecDirectW)")

### Insert data to language table

In [186]:
# Helper function to insert unique company names and generate new IDs
def insert_company(name):
    # Check if the company name already exists in the table
    cursor.execute("SELECT [company_id] FROM [Company] WHERE [name] = ?", name)
    existing_company = cursor.fetchone()

    if existing_company:
        # company already exists, return its ID
        return existing_company[0] 
    else:
        # company doesn't exist, insert it and get the generated ID
        cursor.execute("INSERT INTO [Company] ([name]) VALUES (?)", name)
        conn.commit()  # Commit the insertion to get the generated ID

        # Retrieve the newly generated company_id
        cursor.execute("SELECT @@IDENTITY")
        company_id = cursor.fetchone()[0]

        return company_id

# Insert data into tables
for index, row in df.iterrows():
    # Insert into 'company' table and get the company_id
    company_id = insert_company(row['Company'])

ProgrammingError: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'name'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Statement(s) could not be prepared. (8180)")

In [ ]:
cursor.execute("""
    CREATE TABLE [Location] (
      [country_id] int IDENTITY(1,1) PRIMARY KEY,
      [is_remote] int,
      [Country] varchar(155),
    );
""")
conn.commit()

### Insert data to license table

In [10]:
# Helper function to insert unique Location names and generate new IDs
def insert_Location(is_remote, country):
    # Check if the location name already exists in the table
    cursor.execute("SELECT [country_id] FROM [Location] WHERE [Country] = ?", country)
    existing_Location = cursor.fetchone()

    if existing_Location:
        # License already exists, return its ID
        return existing_Location[0]
    else:
        # Location doesn't exist, insert it and get the generated ID
        cursor.execute("INSERT INTO [Location] ([is_remote], [Country]) VALUES (?, ?)", is_remote, country)
        conn.commit()  # Commit the insertion to get the generated ID

        # Retrieve the newly generated country_id
        cursor.execute("SELECT @@IDENTITY")
        country_id = cursor.fetchone()[0]

        return country_id

# Insert data into 'Location' table
for index, row in df.iterrows():
    # Insert into 'Location' table and get the licence_id
    country_id = insert_Location(row['is_remote'], row['country'])


In [162]:
cursor.execute("""
    CREATE TABLE [contribution] (
      [countrubition_id] INT IDENTITY(1,1) PRIMARY KEY,
      [contrubitor_name] varchar(255)
    );
""")

In [163]:
# Helper function to insert contribution data
def insert_contribution(name_of_contributor):
    # Check if the contributor already exists in the table
    cursor.execute("SELECT COUNT(*) FROM [contribution] WHERE [contrubitor_name] = ?", name_of_contributor)
    count = cursor.fetchone()[0]

    if count == 0:
        # Contributor doesn't exist, insert the data
        cursor.execute("INSERT INTO [contribution] ([contrubitor_name]) VALUES (?)", name_of_contributor)
        conn.commit()

# Insert data into 'contribution' table
unique_contributors = set()  # To store unique contributor names

for index, row in df.iterrows():
    # Assuming 'contributors' column contains a list of dictionaries with 'name_of_contributor' and 'num_of_contributions'
    contributors_list = eval(row['contributors'])

    for contributor in contributors_list:
        # Extract contributor data from the dictionary
        name_of_contributor = contributor['name_of_contributor']

        # Insert into 'contribution' table if it's a unique contributor
        if name_of_contributor not in unique_contributors:
            insert_contribution(name_of_contributor)
            unique_contributors.add(name_of_contributor)


In [164]:
# Fetch data from 'contribution' table
cursor.execute("SELECT *  FROM [contribution]")
rows = cursor.fetchall()

# Print each row of data
for row in rows:
    contribution_id = row[0]
   
    contrubitor_name = row[1]

    print(f"Contribution ID: {contribution_id}, Contributor Name: {contrubitor_name}")


Contribution ID: 1, Contributor Name: nekohasekai
Contribution ID: 2, Contributor Name: H1JK
Contribution ID: 3, Contributor Name: dyhkwong
Contribution ID: 4, Contributor Name: renovate[bot]
Contribution ID: 5, Contributor Name: arm64v8a
Contribution ID: 6, Contributor Name: 0x7d274284
Contribution ID: 7, Contributor Name: mazzz1y
Contribution ID: 8, Contributor Name: reeceyng
Contribution ID: 9, Contributor Name: database64128
Contribution ID: 10, Contributor Name: shadow750d6
Contribution ID: 11, Contributor Name: zakuwaki
Contribution ID: 12, Contributor Name: drprofesq
Contribution ID: 13, Contributor Name: XYenon
Contribution ID: 14, Contributor Name: Dreamacro
Contribution ID: 15, Contributor Name: liaoliaots
Contribution ID: 16, Contributor Name: Fei1Yang
Contribution ID: 17, Contributor Name: everyx
Contribution ID: 18, Contributor Name: zzzgydi
Contribution ID: 19, Contributor Name: H3arn
Contribution ID: 20, Contributor Name: Larvan2
Contribution ID: 21, Contributor Name: Sk

In [165]:

# Get the unique contributor names
unique_contributor_names = df['contributors'].apply(eval).apply(lambda x: [item['name_of_contributor'] for item in x]).explode().unique()

# Calculate the size of unique contributor names
num_unique_contributors = len(unique_contributor_names)

# Print the result
print(f"Number of unique contributor names: {num_unique_contributors}")


Number of unique contributor names: 24617


In [166]:
cursor.execute("""
    CREATE TABLE [owner] (
      [owner_id] INT IDENTITY(1,1) PRIMARY KEY,
      [owner_name] varchar(50),
      [owner_type] varchar(50)
    );
""")
conn.commit()

In [167]:
# Helper function to insert unique owner names and generate new IDs
def insert_owner(owner_name, owner_type):
    # Check if the owner name already exists in the table
    cursor.execute("SELECT [owner_id] FROM [owner] WHERE [owner_name] = ?", owner_name)
    existing_owner = cursor.fetchone()

    if existing_owner:
        # Owner already exists, return its ID
        return existing_owner[0]
    else:
        # Owner doesn't exist, insert it and get the generated ID
        cursor.execute("INSERT INTO [owner] ([owner_name], [owner_type]) VALUES (?, ?)", owner_name, owner_type)
        conn.commit()  # Commit the insertion to get the generated ID

        # Retrieve the newly generated owner_id
        cursor.execute("SELECT @@IDENTITY")
        owner_id = cursor.fetchone()[0]

        return owner_id

# Insert data into tables
for index, row in df.iterrows():
    # Extract owner data from the DataFrame
    owner_name = row['owner name']
    owner_type = row['owner type']

    # Insert into 'owner' table and get the owner_id
    owner_id = insert_owner(owner_name, owner_type)


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16156 entries, 0 to 16155
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           16156 non-null  int64  
 1   full name                    16156 non-null  object 
 2   name                         16156 non-null  object 
 3   url                          16156 non-null  object 
 4   description                  16156 non-null  object 
 5   stars                        16156 non-null  int64  
 6   language                     16156 non-null  object 
 7   forks                        16156 non-null  int64  
 8   open issues                  16156 non-null  int64  
 9   owner name                   16156 non-null  object 
 10  owner type                   16156 non-null  object 
 11  owner site admin             16156 non-null  bool   
 12  license name                 16156 non-null  object 
 13  license key     

In [27]:
# Convert the 'creation_date' column to datetime type
df['creation date'] = pd.to_datetime(df['creation date'])

# Extract only the date part (YYYY-MM-DD) and save it back to the 'creation_date' column
df['creation date'] = df['creation date'].dt.date

In [28]:
# Convert the 'creation_date' column to datetime type
df['updated at'] = pd.to_datetime(df['updated at'])

# Extract only the date part (YYYY-MM-DD) and save it back to the 'creation_date' column
df['updated at'] = df['updated at'].dt.date

In [29]:
# Convert the 'creation_date' column to datetime type
df['pushed at'] = pd.to_datetime(df['pushed at'])

# Extract only the date part (YYYY-MM-DD) and save it back to the 'creation_date' column
df['pushed at'] = df['pushed at'].dt.date

In [37]:
cursor.execute("""
    CREATE TABLE [repository] (
      [id] int,
      [full_name] varchar(255),
      [name] varchar(100),
      [url] varchar(150),
      [description] varchar(4000),
      [stars] int,
      [forks] int,
      [open_issues] int,
      [creation_date] date,
      [update] date,
      [push] date,
      [size] int,
      [score] float,
      [owner_id] int,
      [licence_id] int,
      [language_id] int,
      PRIMARY KEY ([id]),
      CONSTRAINT [FK_repository.owner_id]
        FOREIGN KEY ([owner_id])
          REFERENCES [owner]([owner_id]),
      CONSTRAINT [FK_repository.language_id]
        FOREIGN KEY ([language_id])
          REFERENCES [language]([language_id]),
      CONSTRAINT [FK_repository.licence_id]
        FOREIGN KEY ([licence_id])
          REFERENCES [licence]([licence_id])
    );
""")
conn.commit()

In [ ]:
cursor.execute("""

    CREATE TABLE [Job Offre] (
      [job_id] int IDENTITY(1,1),
      [Job Title] varchar(455),
      [Job Type] varchar(55),
      [Experience level] varchar(55),
      [Job_category] varchar(55),
      [company_id] int,
      [country_id] int,
      [salary] int,
      [the salary may go beyond] int,
      [there are additional conditions or footnotes] int,
      [Negociable] int,
      PRIMARY KEY ([job_id]),
      CONSTRAINT [FK_Job Offre.country_id]
        FOREIGN KEY ([country_id])
          REFERENCES [Location]([country_id]),
      CONSTRAINT [FK_Job Offre.company_id]
        FOREIGN KEY ([company_id])
          REFERENCES [Company]([company_id])
    );
""")
conn.commit()

In [42]:
# Helper function to insert unique owner names and generate new IDs
def insert_owner(owner_name, owner_type):
    # Check if the owner name already exists in the table
    cursor.execute("SELECT [owner_id] FROM [owner] WHERE [owner_name] = ?", owner_name)
    existing_owner = cursor.fetchone()

    if existing_owner:
        # Owner already exists, return its ID
        return existing_owner[0]
    else:
        # Owner doesn't exist, insert it and get the generated ID
        cursor.execute("INSERT INTO [owner] ([owner_name], [owner_type]) VALUES (?, ?)", owner_name, owner_type)
        conn.commit()  # Commit the insertion to get the generated ID

        # Retrieve the newly generated owner_id
        cursor.execute("SELECT @@IDENTITY")
        owner_id = cursor.fetchone()[0]

        return owner_id

# Helper function to convert date string to SQL Server compatible format
def convert_to_sql_server_date(date_obj):
    if isinstance(date_obj, str):
        try:
            date_obj = datetime.datetime.strptime(date_obj, '%Y-%m-%d')
        except ValueError:
            return None  # Handle invalid date format gracefully or raise an exception as needed
    return date_obj.strftime('%Y-%m-%d')

# Helper function to insert data into the 'repository' table
def insert_repository(repo_id, full_name, name, url, description, stars, forks, open_issues, creation_date, update, push, size, score, owner_id, licence_id, language_id):
    cursor.execute("INSERT INTO [repository] ([id], [full_name], [name], [url], [description], [stars], [forks], [open_issues], [creation_date], [update], [push], [size], [score], [owner_id], [licence_id], [language_id]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                   repo_id, full_name, name, url, description, stars, forks, open_issues, creation_date, update, push, size, score, owner_id, licence_id, language_id)
    conn.commit()

# Insert data into 'repository' table
for index, row in df.iterrows():
    repo_id = row['id']
    full_name = row['full name']
    name = row['name']
    url = row['url']
    description = row['description']
    stars = row['stars']
    forks = row['forks']
    open_issues = row['open issues']
    creation_date = row['creation date']
    update = row['updated at']
    push = row['pushed at']
    size = row['size']
    score = row['score']
    owner_name = row['owner name']
    owner_type = row['owner type']
    licence_name = row['license name']
    language_name = row['language']

    # Insert into 'owner' table and get the owner_id
    owner_id = insert_owner(owner_name, owner_type)

    # Convert date strings to SQL Server compatible format
    creation_date = convert_to_sql_server_date(creation_date)
    update = convert_to_sql_server_date(update)
    push = convert_to_sql_server_date(push)

    # Retrieve licence_id and language_id from their respective tables based on the provided licence_key and language_name
    cursor.execute("SELECT [licence_id] FROM [licence] WHERE [licence_name] = ?", licence_name)
    licence_id = cursor.fetchone()[0]

    cursor.execute("SELECT [language_id] FROM [language] WHERE [language_name] = ?", language_name)
    language_id = cursor.fetchone()[0]

    # Insert into 'repository' table and get the repo_id
    insert_repository(repo_id, full_name, name, url, description, stars, forks, open_issues, creation_date, update, push, size, score, owner_id, licence_id, language_id)



In [22]:
# Helper function to insert unique job categories and generate new IDs
def insert_job_category(job_category):
    # Check if the job category already exists in the table
    cursor.execute("SELECT [job_category_id] FROM [Job Categories] WHERE [job_category] = ?", job_category)
    existing_job_category = cursor.fetchone()

    if existing_job_category:
        # job category already exists, return its ID
        return existing_job_category[0]
    else:
        # job category doesn't exist, insert it and get the generated ID
        cursor.execute("INSERT INTO [Job Categories] ([job_category]) VALUES (?)", job_category)
        conn.commit()  # Commit the insertion to get the generated ID

        # Retrieve the newly generated job_category_id
        cursor.execute("SELECT @@IDENTITY")
        job_category_id = cursor.fetchone()[0]

        return job_category_id

In [25]:
df.head()

,Company,Job Title,Location,Job Type,Experience level,Salary,Requirment of the company,Facilities,the salary may go beyond,there are additional conditions or footnotes,Negociable,country,is_remote,Job_category
0,SGS,Clinical Data Analyst,"Richardson, TX, United States",Full Time,Entry-level,48000.0,"Computer Science,Data quality,Genetics,Mathema...",Not-Specified,1,1,0,United States,0,Data Science
1,Ocorian,AML/CFT & Data Analyst,"Ebène, Mauritius",Full Time,Entry-level,48000.0,"Agile,Data management,Finance,Security,,",Not-Specified,1,1,0,Mauritius,0,AI
2,Cricut,Machine Learning Engineer,"South Jordan, UT, United States",Full Time,Not specified,90000.0,"Agile,Architecture,AWS,Computer Science,Comput...",Career development,1,1,0,United States,0,AI
3,Bosch Group,Application Developer & Data Analyst,"Nonantola, Italy",Full Time,Entry-level,48000.0,"Engineering,Industrial,Oracle,Power BI,R,R&D",Not-Specified,1,1,0,Italy,0,Data Science
4,Publicis Groupe,Data Engineer Full time (Public Sector) USA,"Arlington, VA, United States",Full Time,Mid-level,108000.0,"AWS,Azure,Computer Science,Consulting,Dataflow...","Flex hours,Flex vacation,Parental leave,Unlimi...",1,0,0,United States,0,Data Science


In [23]:
# Insert data into 'Job Offre' table
unique_job_categories = set()  # To store unique job categories

In [40]:


for index, row in df.iterrows():
    # Extract job data from row
    job_title = row['Job Title']
    job_type = row['Job Type']
    experience_level = row['Experience level']
    job_category = row['Job_category']
    company_name = row['Company']
    country_name = row['country']
    salary = row['Salary']
    the_salary_may_go_beyond = row['the salary may go beyond']
    additional_conditions_or_footnotes = row['there are additional conditions or footnotes']
    negotiable = row['Negociable']

    # Insert company and location data, and get their IDs
    company_id = insert_Company(name)
    country_id = insert_Location(False, country)

    # Insert job category data, and get its ID
    job_category_id = insert_job_category(job_category)

    # Insert job data into 'Job Offre' table
    cursor.execute("""
        INSERT INTO [Job Offre] ([Job Title], [Job Type], [Experience level], [Job_category], [company_id], [country_id],
                                 [salary], [the salary may go beyond], [there are additional conditions or footnotes], [Negociable])
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, job_title, job_type, experience_level, job_category_id, company_id, country_id, salary, the_salary_may_go_beyond,
                   additional_conditions_or_footnotes, negotiable)

    conn.commit()

NameError: name 'insert_Company' is not defined

In [179]:
cursor.execute("""
    CREATE TABLE [topics] (
      [topic_id] INT IDENTITY(1,1) PRIMARY KEY,
      [topic_name] varchar(100)
    );
""")
conn.commit()

In [ ]:
cursor.execute("""
    CREATE TABLE [Facilities] (
      [facility_id] int IDENTITY(1,1) PRIMARY KEY,
      [facility] varchar(150),
    );
""")
conn.commit()

In [16]:
# Helper function to insert unique facility names and generate new IDs
def insert_facility(facility):
    # Check if the facility already exists in the table
    cursor.execute("SELECT [facility_id] FROM [Facilities] WHERE [facility] = ?", facility)
    existing_facility = cursor.fetchone()

    if existing_facility:
        # facility already exists, return its ID
        return existing_facility[0]
    else:
        # facility doesn't exist, insert it and get the generated ID
        cursor.execute("INSERT INTO [Facilities] ([facility]) VALUES (?)", facility)
        conn.commit()  # Commit the insertion to get the generated ID

        # Retrieve the newly generated facility_id
        cursor.execute("SELECT @@IDENTITY")
        facility_id = cursor.fetchone()[0]

        return facility_id

# Insert data into 'Facilities' table
unique_facilities = set()  # To store unique facility

for index, row in df.iterrows():
    # Assuming 'Facilities' column contains a comma-separated string of facilities
    facilities_list = row['Facilities'].split(',')

    for facility in facilities_list:
        # Extract and clean facility
        facility = facility.strip()

        # Insert into 'Facilities' table if it's a unique facility
        if facility not in unique_facilities:
            insert_facility(facility)
            unique_facilities.add(facility)


In [43]:
cursor.execute("""
    CREATE TABLE [repo-contrubition] (
      [repository_id] int,
      [contrubition_id] int,
      [number_of_contrubitions] int,
      CONSTRAINT [FK_repo-contrubition.contrubition_id]
        FOREIGN KEY ([contrubition_id])
          REFERENCES [contribution]([countrubition_id]),
      CONSTRAINT [FK_repo-contrubition.repository_id]
        FOREIGN KEY ([repository_id])
          REFERENCES [repository]([id])
    );
""")
conn.commit()

In [44]:
# Helper function to insert data into the 'repo-contrubition' table
def insert_repo_contrubition(repository_id, contrubition_id, num_contributions):
    cursor.execute("INSERT INTO [repo-contrubition] ([repository_id], [contrubition_id], [number_of_contrubitions]) VALUES (?, ?, ?)",
                   repository_id, contrubition_id, num_contributions)
    conn.commit()

# Insert data into 'repo-contrubition' table
for index, row in df.iterrows():
    repo_id = row['id']
    # Assuming 'contributors' column contains a list of dictionaries with 'name_of_contributor' and 'num_of_contributions'
    contributors_list = eval(row['contributors'])

    for contributor in contributors_list:
        # Extract contributor data from the dictionary
        name_of_contributor = contributor['name_of_contributor']
        num_of_contributions = contributor['num_of_contributions']

        # Find the contrubition_id from the 'contribution' table based on the 'name_of_contributor'
        cursor.execute("SELECT [countrubition_id] FROM [contribution] WHERE [contrubitor_name] = ?", name_of_contributor)
        contrubition_id = cursor.fetchone()

        if contrubition_id:
            contrubition_id = contrubition_id[0]
            # Insert into 'repo-contrubition' table
            insert_repo_contrubition(repo_id, contrubition_id, num_of_contributions)


In [45]:
cursor.execute("""
    CREATE TABLE [repo-topic] (
      [topic_id] int,
      [repository_id] int,
      CONSTRAINT [FK_repo-topic.repository_id]
        FOREIGN KEY ([repository_id])
          REFERENCES [repository]([id]),
      CONSTRAINT [FK_repo-topic.topic_id]
        FOREIGN KEY ([topic_id])
          REFERENCES [topics]([topic_id])
    );
""")
conn.commit()

In [46]:
# Helper function to get the topic_id for a given topic_name
def get_topic_id(topic_name):
    cursor.execute("SELECT [topic_id] FROM [topics] WHERE [topic_name] = ?", topic_name)
    topic_id = cursor.fetchone()
    if topic_id:
        return topic_id[0]
    else:
        # Handle the case where the topic_name does not exist in the topics table
        return None

# Helper function to insert data into the 'repo-topic' table
def insert_repo_topic(repository_id, topic_name):
    topic_id = get_topic_id(topic_name)
    if topic_id:
        cursor.execute("INSERT INTO [repo-topic] ([repository_id], [topic_id]) VALUES (?, ?)",
                       repository_id, topic_id)
        conn.commit()

# Insert data into 'repo-topic' table
for index, row in df.iterrows():
    repo_id = row['id']
    # Assuming 'topics' column contains a list of topic names
    topics_list = eval(row['topics'])

    for topic_name in topics_list:
        # Insert into 'repo-topic' table
        insert_repo_topic(repo_id, topic_name)


In [47]:
# Commit the changes and close the connection
conn.commit()
conn.close()

<h1 style="color:RED;"> --------------------------------------------------------------------------------------------------------------------- </h1>


## insert into company table

In [187]:
# Helper function to insert unique company names and generate new IDs
def insert_company(name):
    # Check if the company name already exists in the table
    cursor.execute("SELECT [company_id] FROM [Company] WHERE [company_name] = ?", name)
    existing_company = cursor.fetchone()

    if existing_company:
        # company already exists, return its ID
        return existing_company[0] 
    else:
        # company doesn't exist, insert it and get the generated ID
        cursor.execute("INSERT INTO [Company] ([company_name]) VALUES (?)", name)
        conn.commit()  # Commit the insertion to get the generated ID

        # Retrieve the newly generated company_id
        cursor.execute("SELECT @@IDENTITY")
        company_id = cursor.fetchone()[0]

        return company_id

# Insert data into tables
for index, row in df.iterrows():
    # Insert into 'company' table and get the company_id
    company_id = insert_company(row['Company'])

## insert into Location Table

In [188]:
# Helper function to insert unique Location names and generate new IDs
def insert_Location(country):
    # Check if the location name already exists in the table
    cursor.execute("SELECT [country_id] FROM [Country] WHERE [country_name] = ?", country)
    existing_Location = cursor.fetchone()

    if existing_Location:
        # Country already exists, return its ID
        return existing_Location[0]
    else:
        # Country doesn't exist, insert it and get the generated ID
        cursor.execute("INSERT INTO [Country] ( [country_name]) VALUES (?)", country)
        conn.commit()  # Commit the insertion to get the generated ID

        # Retrieve the newly generated country_id
        cursor.execute("SELECT @@IDENTITY")
        country_id = cursor.fetchone()[0]

        return country_id

# Insert data into 'Country' table
for index, row in df.iterrows():
    # Insert into 'Country' table and get the country_id
    country_id = insert_Location(row['country'])

## insert into facilities

In [189]:
# Helper function to insert unique facility names and generate new IDs
def insert_facility_name(facility_name):
    # Check if the facility already exists in the table
    cursor.execute("SELECT [facility_id] FROM [Facilities] WHERE [facility_name] = ?", facility_name)
    existing_facility_name = cursor.fetchone()

    if existing_facility_name:
        # facility already exists, return its ID
        return existing_facility_name[0]
    else:
        # facility doesn't exist, insert it and get the generated ID
        cursor.execute("INSERT INTO [Facilities] ([facility_name]) VALUES (?)", facility_name)
        conn.commit()  # Commit the insertion to get the generated ID

        # Retrieve the newly generated facility_id
        cursor.execute("SELECT @@IDENTITY")
        facility_id = cursor.fetchone()[0]

        return facility_id

# Insert data into 'Facilities' table
unique_facilities = set()  # To store unique facility_name

for index, row in df.iterrows():
    # Assuming 'Facilities' column contains a comma-separated string of facilities
    facilities_list = row['Facilities'].split(',')

    for facility_name in facilities_list:
        # Extract and clean facility
        facility_name = facility_name.strip()

        # Insert into 'Facilities' table if it's a unique facility_name
        if facility_name not in unique_facilities:
            insert_facility_name(facility_name)
            unique_facilities.add(facility_name)

In [ ]:
## insert into 

In [41]:
df

,Company,Job Title,Location,Job Type,Experience level,Salary,Requirment of the company,Facilities,the salary may go beyond,there are additional conditions or footnotes,Negociable,country,is_remote,Job_category
0,SGS,Clinical Data Analyst,"Richardson, TX, United States",Full Time,Entry-level,48000.0,"Computer Science,Data quality,Genetics,Mathema...",Not-Specified,1,1,0,United States,0,Data Science
1,Ocorian,AML/CFT & Data Analyst,"Ebène, Mauritius",Full Time,Entry-level,48000.0,"Agile,Data management,Finance,Security,,",Not-Specified,1,1,0,Mauritius,0,AI
2,Cricut,Machine Learning Engineer,"South Jordan, UT, United States",Full Time,Not specified,90000.0,"Agile,Architecture,AWS,Computer Science,Comput...",Career development,1,1,0,United States,0,AI
3,Bosch Group,Application Developer & Data Analyst,"Nonantola, Italy",Full Time,Entry-level,48000.0,"Engineering,Industrial,Oracle,Power BI,R,R&D",Not-Specified,1,1,0,Italy,0,Data Science
4,Publicis Groupe,Data Engineer Full time (Public Sector) USA,"Arlington, VA, United States",Full Time,Mid-level,108000.0,"AWS,Azure,Computer Science,Consulting,Dataflow...","Flex hours,Flex vacation,Parental leave,Unlimi...",1,0,0,United States,0,Data Science
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,CCRi,"Application Integration Engineer, Computer Vis...","Chantilly, Virginia, United States",Full Time,Mid-level,113000.0,"Agile,Angular,APIs,Architecture,AWS,Azure","401(k) matching,Career development,Flex hours,...",1,0,0,United States,0,Other
2991,Publicis Groupe,"Associate Director, Data Science","New York City, United States",Full Time,Mid-level,106000.0,"Bayesian,Classification,Clustering,Data analys...","Career development,Health care",1,0,0,United States,0,Data Science
2992,DoorDash,"Senior Software Engineer, Machine Learning - A...","Sunnyvale, CA; San Francisco, CA; New York",Full Time,Senior-level,176000.0,"Computer Science,Data analysis,Engineering,Exc...","401(k) matching,Career development,Equity,Insu...",1,0,0,Other,0,AI
2993,Western Digital,Data Scientist - New College Graduate,"Biñan, Philippines",Full Time,Entry-level,39000.0,"APIs,Clustering,Computer Science,Data visualiz...",Career development,1,1,0,Philippines,0,Data Science


In [190]:
# Helper function to insert unique Requirment names and generate new IDs
def insert_requirment_name(requirment_name):
    # Check if the Requirment already exists in the table
    cursor.execute("SELECT [requirment_id] FROM [Requirment of the company ] WHERE [requirment_name] = ?", requirment_name)
    existing_requirment_name = cursor.fetchone()

    if existing_requirment_name:
        # Requirment already exists, return its ID
        return existing_requirment_name[0]
    else:
        # Requirment doesn't exist, insert it and get the generated ID
        cursor.execute("INSERT INTO [Requirment of the company] ([requirment_name]) VALUES (?)", requirment_name)
        conn.commit()  # Commit the insertion to get the generated ID

        # Retrieve the newly generated Requirment_id
        cursor.execute("SELECT @@IDENTITY")
        requirment_id = cursor.fetchone()[0]

        return requirment_id

# Insert data into 'Requirment' table
unique_requirment_name = set()  # To store unique Requirment

for index, row in df.iterrows():
    # Assuming 'Requirment' column contains a comma-separated string of Requirment
    Requirment_list = row['Requirment of the company '].split(',')

    for requirment_name in Requirment_list:
        # Extract and clean requirment
        requirment_name = requirment_name.strip()

        # Insert into 'Requirment' table if it's a unique Requirment
        if requirment_name not in unique_requirment_name:
            insert_requirment_name(requirment_name)
            unique_requirment_name.add(requirment_name)

In [191]:
df.isnull().sum()

Company                                          0
Job Title                                        0
Location                                         0
Job Type                                         0
Experience level                                 0
Salary                                          99
Requirment of the company                        0
Facilities                                       0
the salary may go beyond                         0
there are additional conditions or footnotes     0
Negociable                                       0
country                                          0
is_remote                                        0
Job_category                                     0
dtype: int64

In [192]:
cn = df.loc[(df['Salary'].isna())]
df = df.drop(cn.index)

In [193]:
# Helper function to insert data into the 'Job Offre' table
def insert_job_offre(job_title, job_type, job_category, company_id, country_id, salary, is_remote, beyond, conditions, negociable):
    cursor.execute("INSERT INTO [Job Offre] ([Job Title], [Job Type], [Job_category], [company_id], [country_id], [Salary], [is_remote], [beyond], [conditions], [Negociable]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                   job_title, job_type, job_category, company_id, country_id, salary, is_remote, beyond, conditions, negociable)
    conn.commit()

# Insert data into 'Job Offre' table
for index, row in df.iterrows():
    job_title = row['Job Title']
    job_type = row['Job Type']
    job_category = row['Job_category']
    company_name = row['Company']
    country_name = row['country']
    salary = row['Salary']
    is_remote = row['is_remote']
    beyond = row['the salary may go beyond']
    conditions = row['there are additional conditions or footnotes']
    negociable = row['Negociable']

    # Retrieve the company_id and country_id from their respective tables using their names
    cursor.execute("SELECT [company_id] FROM [Company] WHERE [company_name] = ?", company_name)
    company_id = cursor.fetchone()[0]

    cursor.execute("SELECT [country_id] FROM [Country] WHERE [country_name] = ?", country_name)
    country_id = cursor.fetchone()[0]
#     print(row)
    # Insert into 'Job Offre' table
    insert_job_offre(job_title, job_type, job_category, company_id, country_id, salary, is_remote, beyond, conditions, negociable)

In [ ]:
------------------------------------------------------------

In [160]:
df

,Company,Job Title,Location,Job Type,Experience level,Salary,Requirment of the company,Facilities,the salary may go beyond,there are additional conditions or footnotes,Negociable,country,is_remote,Job_category
0,SGS,Clinical Data Analyst,"Richardson, TX, United States",Full Time,Entry-level,48000.0,"Computer Science,Data quality,Genetics,Mathema...",Not-Specified,1,1,0,United States,0,Data Science
1,Ocorian,AML/CFT & Data Analyst,"Ebène, Mauritius",Full Time,Entry-level,48000.0,"Agile,Data management,Finance,Security,,",Not-Specified,1,1,0,Mauritius,0,AI
2,Cricut,Machine Learning Engineer,"South Jordan, UT, United States",Full Time,Not specified,90000.0,"Agile,Architecture,AWS,Computer Science,Comput...",Career development,1,1,0,United States,0,AI
3,Bosch Group,Application Developer & Data Analyst,"Nonantola, Italy",Full Time,Entry-level,48000.0,"Engineering,Industrial,Oracle,Power BI,R,R&D",Not-Specified,1,1,0,Italy,0,Data Science
4,Publicis Groupe,Data Engineer Full time (Public Sector) USA,"Arlington, VA, United States",Full Time,Mid-level,108000.0,"AWS,Azure,Computer Science,Consulting,Dataflow...","Flex hours,Flex vacation,Parental leave,Unlimi...",1,0,0,United States,0,Data Science
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,CCRi,"Application Integration Engineer, Computer Vis...","Chantilly, Virginia, United States",Full Time,Mid-level,113000.0,"Agile,Angular,APIs,Architecture,AWS,Azure","401(k) matching,Career development,Flex hours,...",1,0,0,United States,0,Other
2991,Publicis Groupe,"Associate Director, Data Science","New York City, United States",Full Time,Mid-level,106000.0,"Bayesian,Classification,Clustering,Data analys...","Career development,Health care",1,0,0,United States,0,Data Science
2992,DoorDash,"Senior Software Engineer, Machine Learning - A...","Sunnyvale, CA; San Francisco, CA; New York",Full Time,Senior-level,176000.0,"Computer Science,Data analysis,Engineering,Exc...","401(k) matching,Career development,Equity,Insu...",1,0,0,Other,0,AI
2993,Western Digital,Data Scientist - New College Graduate,"Biñan, Philippines",Full Time,Entry-level,39000.0,"APIs,Clustering,Computer Science,Data visualiz...",Career development,1,1,0,Philippines,0,Data Science


In [202]:
# Helper function to insert data into the 'facility_job' table
def insert_facility_job(job_id, facility_id):
    cursor.execute("INSERT INTO [facility-job] ([job_id], [facility_id]) VALUES (?, ?)",
                   job_id, facility_id)
    conn.commit()
    
# Insert data into 'facility_job' table
for index, row in df.iterrows():
    job_title = row['Job Title']
    facility_name = row['Facilities']

    # Find the job_id from the 'Job Offer' table based on the 'job_title'
    cursor.execute("SELECT [job_id] FROM [Job Offre] WHERE [Job Title] = ?", job_title)
    job_id = cursor.fetchone()

    # Find the facility_id from the 'Facilities' table based on the 'facility_name'
    cursor.execute("SELECT [facility_id] FROM [Facilities] WHERE [facility_name] = ?", facility_name)
    facility_id = cursor.fetchone()

    if job_id and facility_id:
        job_id = job_id[0]
        facility_id = facility_id[0]
        # Insert into 'facility_job' table
        insert_facility_job(job_id, facility_id)

In [203]:
df

,Company,Job Title,Location,Job Type,Experience level,Salary,Requirment of the company,Facilities,the salary may go beyond,there are additional conditions or footnotes,Negociable,country,is_remote,Job_category
0,SGS,Clinical Data Analyst,"Richardson, TX, United States",Full Time,Entry-level,48000.0,"Computer Science,Data quality,Genetics,Mathema...",Not-Specified,1,1,0,United States,0,Data Science
1,Ocorian,AML/CFT & Data Analyst,"Ebène, Mauritius",Full Time,Entry-level,48000.0,"Agile,Data management,Finance,Security,,",Not-Specified,1,1,0,Mauritius,0,AI
2,Cricut,Machine Learning Engineer,"South Jordan, UT, United States",Full Time,Not specified,90000.0,"Agile,Architecture,AWS,Computer Science,Comput...",Career development,1,1,0,United States,0,AI
3,Bosch Group,Application Developer & Data Analyst,"Nonantola, Italy",Full Time,Entry-level,48000.0,"Engineering,Industrial,Oracle,Power BI,R,R&D",Not-Specified,1,1,0,Italy,0,Data Science
4,Publicis Groupe,Data Engineer Full time (Public Sector) USA,"Arlington, VA, United States",Full Time,Mid-level,108000.0,"AWS,Azure,Computer Science,Consulting,Dataflow...","Flex hours,Flex vacation,Parental leave,Unlimi...",1,0,0,United States,0,Data Science
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,CCRi,"Application Integration Engineer, Computer Vis...","Chantilly, Virginia, United States",Full Time,Mid-level,113000.0,"Agile,Angular,APIs,Architecture,AWS,Azure","401(k) matching,Career development,Flex hours,...",1,0,0,United States,0,Other
2991,Publicis Groupe,"Associate Director, Data Science","New York City, United States",Full Time,Mid-level,106000.0,"Bayesian,Classification,Clustering,Data analys...","Career development,Health care",1,0,0,United States,0,Data Science
2992,DoorDash,"Senior Software Engineer, Machine Learning - A...","Sunnyvale, CA; San Francisco, CA; New York",Full Time,Senior-level,176000.0,"Computer Science,Data analysis,Engineering,Exc...","401(k) matching,Career development,Equity,Insu...",1,0,0,Other,0,AI
2993,Western Digital,Data Scientist - New College Graduate,"Biñan, Philippines",Full Time,Entry-level,39000.0,"APIs,Clustering,Computer Science,Data visualiz...",Career development,1,1,0,Philippines,0,Data Science


In [ ]:
cursor.execute("""
    CREATE TABLE [requirment-job] (
      [job_id] int,
      [requirment_id] int,
      CONSTRAINT [FK_requirment-job.requirment_id]
        FOREIGN KEY ([requirment_id])
          REFERENCES [Requirment of the company]([requirment_id]),
      CONSTRAINT [FK_requirment-job.job_id]
        FOREIGN KEY ([job_id])
          REFERENCES [Job Offre]([job_id])
);
""")
conn.commit()

requirment_job

In [210]:
# Helper function to insert data into the 'requirment_job' table
def insert_requirment_job(job_id, requirment_id):
    cursor.execute("INSERT INTO [requirment-job] ([job_id], [requirment_id]) VALUES (?, ?)",
                   job_id, requirment_id)
    conn.commit()
    
# Insert data into 'requirment_job' table
for index, row in df.iterrows():
    job_title = row['Job Title']
    Requirment_name= row['Requirment of the company ']

    # Find the job_id from the 'Job Offer' table based on the 'job_title'
    cursor.execute("SELECT [job_id] FROM [Job Offre] WHERE [Job Title] = ?", job_title)
    job_id = cursor.fetchone()

    # Find the requirment_id from the 'Requirment of the company' table based on the 'Requirment of the company'
    cursor.execute("SELECT [requirment_id] FROM [Requirment of the company ] WHERE [requirment_name] = ?", (Requirment_name,))
    requirment_id = cursor.fetchone()

    if job_id and requirment_id:
        job_id = job_id[0]
        requirment_id = requirment_id[0]
        # Insert into 'requirment_job' table
        insert_requirment_job(job_id, requirment_id)

In [134]:
# Commit the changes and close the connection
conn.commit()
conn.close()